## Deliverable 2. Create a Customer Travel Destinations Map.

In [37]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [52]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Albany,US,42.6001,-73.9662,82.56,83,100,6.67,overcast clouds
1,1,Ribeira Grande,PT,38.5167,-28.7000,77.40,83,20,5.75,few clouds
2,2,Ostrovnoy,RU,68.0531,39.5131,59.63,60,100,11.52,overcast clouds
3,3,Bluff,NZ,-46.6000,168.3333,34.66,83,7,4.76,clear sky
4,4,Nouakchott,MR,18.0858,-15.9785,85.93,75,15,14.23,few clouds


In [54]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))


What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [55]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)


,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Albany,US,42.6001,-73.9662,82.56,83,100,6.67,overcast clouds
1,1,Ribeira Grande,PT,38.5167,-28.7000,77.40,83,20,5.75,few clouds
4,4,Nouakchott,MR,18.0858,-15.9785,85.93,75,15,14.23,few clouds
10,10,Madaoua,NE,14.0730,5.9600,88.12,45,100,8.34,overcast clouds
14,14,Hithadhoo,MV,-0.6000,73.0833,82.33,68,71,3.53,broken clouds
17,17,Victoria,HK,22.2855,114.1577,79.32,94,100,11.01,light rain
23,23,Buchanan,LR,5.8808,-10.0467,78.80,76,100,10.31,light rain
37,37,Wajir,KE,1.7471,40.0573,87.76,33,75,10.36,broken clouds
40,40,Leon Valley,US,29.4952,-98.6186,81.88,81,75,3.44,broken clouds
41,41,Huangchuan,CN,32.1272,115.0394,82.38,80,100,3.47,moderate rain


In [56]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()


City_ID                280
City                   280
Country                279
Lat                    280
Lng                    280
Max Temp               280
Humidity               280
Cloudiness             280
Wind Speed             280
Current Description    280
dtype: int64

In [57]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
preferred_cities_df1 = preferred_cities_df.dropna()

preferred_cities_df1.count()


City_ID                279
City                   279
Country                279
Lat                    279
Lng                    279
Max Temp               279
Humidity               279
Cloudiness             279
Wind Speed             279
Current Description    279
dtype: int64

In [58]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df1[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Albany,US,82.56,overcast clouds,42.6001,-73.9662,
1,Ribeira Grande,PT,77.40,few clouds,38.5167,-28.7000,
4,Nouakchott,MR,85.93,few clouds,18.0858,-15.9785,
10,Madaoua,NE,88.12,overcast clouds,14.0730,5.9600,
14,Hithadhoo,MV,82.33,broken clouds,-0.6000,73.0833,
17,Victoria,HK,79.32,light rain,22.2855,114.1577,
23,Buchanan,LR,78.80,light rain,5.8808,-10.0467,
37,Wajir,KE,87.76,broken clouds,1.7471,40.0573,
40,Leon Valley,US,81.88,broken clouds,29.4952,-98.6186,
41,Huangchuan,CN,82.38,moderate rain,32.1272,115.0394,


In [ ]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [ ]:
# 7. Drop the rows where there is no Hotel Name.

clean_df = hotel_df.replace(r'^\s*$', np.nan, regex=True)
clean_hotel_df = clean_df.dropna()
clean_hotel_df



In [39]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [49]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Weather Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [51]:
# 11a. Add a marker layer for each city to the map.
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)


# 11b. Display the figure

fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
fig.add_layer(marker_layer)
fig


Figure(layout=FigureLayout(height='420px'))